In [47]:
import numpy as np
import pandas as pd

import sklearn.feature_extraction.text as text
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from textblob import TextBlob
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords



import string
import itertools

import matplotlib.pyplot as plt
import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')   

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
df1 = pd.read_excel("Train.xlsx")

In [49]:
from sklearn.utils import shuffle
df1 = shuffle(df1)

In [50]:
df1

,Exit_Type_Reasons,Parameters
14,Family Issues,Family Problem
0,Termination,Age
19,Inter Org Transfer,Redeployment
2,Decease,Age
11,Distance/Travel time,DistanceFromHome
37,Behavior Issue,supervisor dissatisfaction
23,Better Career Opportunity,growth-career progression
38,Higher studies,higher education
9,Location Issue,DistanceFromHome
17,Marriage,Marriage


In [51]:
df1['Exit_Type_Reasons'] =df1['Exit_Type_Reasons'].apply(lambda x: ' '.join([i.lower() for i in x.split()]))
df1['Exit_Type_Reasons'] =df1['Exit_Type_Reasons'].str.replace(r'[^\w\s]',"")

In [52]:
df1['Exit_Type_Reasons'] =df1['Exit_Type_Reasons'].apply(lambda x: ' '.join([i for i in x.split() if i not in stopword_list]))

In [53]:
df1['Exit_Type_Reasons'] =df1['Exit_Type_Reasons'].apply(lambda x: str(TextBlob(x).correct()))

In [54]:
df1['Exit_Type_Reasons'] =df1['Exit_Type_Reasons'].apply(lambda x:' '.join([Word(i).lemmatize() for i in x.split()]))

In [55]:
df1=df1[df1["Parameters "]  != 'BusinessTravel']


<b> Label encoding

In [113]:
enc = preprocessing.LabelEncoder()
para1= enc.fit_transform(df1['Parameters '])

In [115]:
train_x, valid_x, train_y, valid_y = train_test_split(df1['Exit_Type_Reasons'], para1,stratify=para1, 
                                           cf5         test_size=0.3)

<b>Feautre Extraction

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df1['Exit_Type_Reasons'])


In [ ]:
Model evaluatio

In [ ]:
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [117]:
from sklearn.model_selection import GridSearchCV
clf = LogisticRegression()
lr_params = {'C':[int(x) for x in np.linspace(1,100,10)]}
grid_lr = GridSearchCV(estimator=clf,param_grid=lr_params,cv=2,n_jobs=-1)
grid_lr.fit(xtrain_tfidf,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=2, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1, 12, 23, 34, 45, 56, 67, 78, 89, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [179]:
print(grid_lr.best_params_)


{'C': 12}


In [123]:
final_lr = LogisticRegression(C=12)
final_lr.fit(xtrain_tfidf,train_y)

LogisticRegression(C=12, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [124]:
final_lr_predict = final_lr.predict(xvalid_tfidf)
lr_accuracy = metrics.accuracy_score(final_lr_predict, valid_y)
print ("Logistic Regression > Accuracy: ", lr_accuracy)

Logistic Regression > Accuracy:  0.7692307692307693


In [84]:
final=pd.read_excel("Exit Reasons.xlsx")

In [125]:
para=final_lr.predict(tfidf_vect.transform(final["Exit_Type_Reasons"]))

In [169]:
from collections import Counter
count=Counter(para)
count

Counter({0: 75,
         1: 209,
         2: 2632,
         3: 1987,
         4: 80,
         5: 780,
         6: 780,
         7: 326,
         8: 2443,
         9: 2514,
         10: 396,
         11: 2193})

In [173]:
len(count)

12

In [158]:
d = dict(zip(df1["Parameters "],para1))
d

{'Age': 0,
 'DistanceFromHome': 1,
 'Family Problem': 2,
 'High pressure job': 3,
 'Marriage': 4,
 'Medical Problems ': 5,
 'PerformanceRating': 6,
 'Redeployment': 7,
 'communication gap': 8,
 'growth-career progression': 9,
 'higher education': 10,
 'supervisor dissatisfaction': 11}

In [170]:
d.keys()

dict_keys(['Family Problem', 'Age', 'Redeployment', 'DistanceFromHome', 'supervisor dissatisfaction', 'growth-career progression', 'higher education', 'Marriage', 'PerformanceRating', 'communication gap', 'High pressure job', 'Medical Problems '])

In [176]:
paramters=[]
Reasons=[]
for i in range(len(count)):
  for key, value in d.items():
    if value==i:
      paramters.append(key)
      Reasons.append(count[i])


In [177]:
data=pd.DataFrame(list(zip(paramters,Reasons)), 
               columns =["paramters",'No. of Reasons'])

In [178]:
data

,paramters,No. of Reasons
0,Age,75
1,DistanceFromHome,209
2,Family Problem,2632
3,High pressure job,1987
4,Marriage,80
5,Medical Problems,780
6,PerformanceRating,780
7,Redeployment,326
8,communication gap,2443
9,growth-career progression,2514
